In [3]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)




In [4]:
%cd path/to/Text-Summarizer
sys.path.append(os.path.abspath("./"))


[WinError 3] The system cannot find the path specified: 'path/to/Text-Summarizer'
c:\Users\Tanveer\Text-Summarizer\research


In [5]:
%pwd

'c:\\Users\\Tanveer\\Text-Summarizer\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\Tanveer\\Text-Summarizer'

In [69]:
import sys
print(sys.path)


['c:\\Users\\Tanveer\\anaconda3\\envs\\texts\\python38.zip', 'c:\\Users\\Tanveer\\anaconda3\\envs\\texts\\DLLs', 'c:\\Users\\Tanveer\\anaconda3\\envs\\texts\\lib', 'c:\\Users\\Tanveer\\anaconda3\\envs\\texts', '', 'c:\\Users\\Tanveer\\anaconda3\\envs\\texts\\lib\\site-packages', 'c:\\Users\\Tanveer\\anaconda3\\envs\\texts\\lib\\site-packages\\win32', 'c:\\Users\\Tanveer\\anaconda3\\envs\\texts\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Tanveer\\anaconda3\\envs\\texts\\lib\\site-packages\\Pythonwin', 'c:\\Users\\Tanveer\\Text-Summarizer', 'c:\\Users\\Tanveer\\Text-Summarizer\\research', 'c:\\Users\\Tanveer', 'c:\\Users\\Tanveer\\Text-Summarizer', 'c:\\Users', 'c:\\Users\\Tanveer', 'c:\\', 'c:\\Users', 'c:\\']


In [70]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_files: Path
    unzip_dir: Path

In [71]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
from textSummarizer.config.configuration import ConfigurationManager


In [72]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [73]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [74]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [3]:
import os
import urllib.request
import zipfile

# Configuration section
root_dir = "artifacts/data_ingestion"
source_URL = "https://github.com/entbappy/Branching-tutorial/raw/refs/heads/master/summarizer-data.zip"  
local_data_file = "artifacts/data.zip"
unzip_dir = "artifacts/unzipped_data"

# Create necessary directories
os.makedirs(os.path.dirname(local_data_file), exist_ok=True)
os.makedirs(unzip_dir, exist_ok=True)

# Download the ZIP file
def download_file(url, save_path):
    print("Downloading file...")
    urllib.request.urlretrieve(url, save_path)
    print("Download completed.")

# Extract the ZIP file
def extract_zip_file(zip_path, extract_to):
    print("Extracting ZIP...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("Extraction completed.")

# Run the process
try:
    download_file(source_URL, local_data_file)
    extract_zip_file(local_data_file, unzip_dir)
except Exception as e:
    print("Error:", e)


Download completed.
Extracting ZIP...
Extraction completed.


In [12]:
import os
import urllib.request
import zipfile

class DataIngestionConfig:
    def __init__(self, root_dir,source_url, local_data_file, unzip_dir):
        self.root_dir = root_dir
        self.source_url = source_url
        self.local_zip_path = local_data_file
        self.unzip_dir = unzip_dir

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        os.makedirs(os.path.dirname(self.config.local_zip_path), exist_ok=True)
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        os.makedirs(self.config.root_dir, exist_ok=True)

    def download_file(self):
        print("🔽 Downloading file...")
        urllib.request.urlretrieve(self.config.source_url, self.config.local_zip_path)
        print(f"✅ Downloaded to: {self.config.local_zip_path}")

    def extract_zip_file(self):
        print("🗜️ Extracting ZIP file...")
        with zipfile.ZipFile(self.config.local_zip_path, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        print(f"✅ Extracted to: {self.config.unzip_dir}")

        # List all extracted files
        print("📂 Extracted Files:")
        for root, dirs, files in os.walk(self.config.unzip_dir):
            for file in files:
                print("  -", os.path.join(root, file))

# Example usage
if __name__ == "__main__":
    config = DataIngestionConfig(
        root_dir= "artifacts/data_ingestion",
        source_url= "https://github.com/entbappy/Branching-tutorial/raw/refs/heads/master/summarizer-data.zip ",
        local_data_file= "artifacts/data.zip",
        unzip_dir= "artifacts/unzipped_data"
    )
    
    ingestion = DataIngestion(config)
    
    try:
        ingestion.download_file()
        ingestion.extract_zip_file()
    except Exception as e:
        print("❌ Error occurred:", e)
print("✅ Does data.zip exist?", os.path.exists(config.local_zip_path))
print("✅ Does unzipped_data/ exist?", os.path.exists(config.unzip_dir))




🔽 Downloading file...
✅ Downloaded to: artifacts/data.zip
🗜️ Extracting ZIP file...
✅ Extracted to: artifacts/unzipped_data
📂 Extracted Files:
  - artifacts/unzipped_data\samsum-test.csv
  - artifacts/unzipped_data\samsum-train.csv
  - artifacts/unzipped_data\samsum-validation.csv
  - artifacts/unzipped_data\samsum_dataset\dataset_dict.json
  - artifacts/unzipped_data\samsum_dataset\test\data-00000-of-00001.arrow
  - artifacts/unzipped_data\samsum_dataset\test\dataset_info.json
  - artifacts/unzipped_data\samsum_dataset\test\state.json
  - artifacts/unzipped_data\samsum_dataset\train\data-00000-of-00001.arrow
  - artifacts/unzipped_data\samsum_dataset\train\dataset_info.json
  - artifacts/unzipped_data\samsum_dataset\train\state.json
  - artifacts/unzipped_data\samsum_dataset\validation\data-00000-of-00001.arrow
  - artifacts/unzipped_data\samsum_dataset\validation\dataset_info.json
  - artifacts/unzipped_data\samsum_dataset\validation\state.json
✅ Does data.zip exist? True
✅ Does unzi